Unit 2 Excercise

Author: Zuriel Eliazar D. Calix

Date: September 12, 2025



In [91]:
import numpy as np

In [92]:
class Dense_Layer:
    """
    Custom dense layer class
    """

    def __init__(self, inputs, weights, bias, activation=None):
        # (a) Setup: accept inputs, weights, and bias
        self.inputs = np.asarray(inputs, dtype=float).ravel()
        self.weights = np.asarray(weights, dtype=float)
        self.bias = np.asarray(bias, dtype=float).ravel()
        self.activation = activation
        self.z = None
        self.a = None

    def forward(self):
        # (b) Weighted sum + bias
        x = self.inputs
        W = self.weights
        b = self.bias

        if W.ndim == 1:
            z = np.dot(x, W) + (b.item() if b.size == 1 else b)
        elif W.ndim == 2:
            if W.shape[1] == x.size:    # teacher-style layout
                z = W.dot(x)
            elif W.shape[0] == x.size:  # alt layout
                z = x.dot(W)
            else:
                raise ValueError(f"Incompatible shape {W.shape} for input size {x.size}")
            if b.size == 1:
                z = z + b.item()
            elif b.size == z.size:
                z = z + b
            else:
                raise ValueError(f"Bias shape {b.shape} incompatible with z shape {z.shape}")
        else:
            raise ValueError("Weights must be 1-D or 2-D")

        self.z = np.asarray(z, dtype=float)

        # (c) Activation function
        if self.activation is None:
            self.a = self.z
        elif self.activation.lower() == 'relu':
            self.a = np.maximum(0.0, self.z)
        elif self.activation.lower() == 'sigmoid':
            z_clip = np.clip(self.z, -500, 500)
            self.a = 1.0 / (1.0 + np.exp(-z_clip))
        else:
            raise ValueError("Unsupported activation. Use None, 'relu', or 'sigmoid'.")

        return self.a

    @staticmethod
    def loss(predicted, target, eps=1e-12, reduction='mean'):
        # (d) Loss function: binary cross-entropy
        p = np.clip(np.asarray(predicted, dtype=float).ravel(), eps, 1.0 - eps)
        t = np.asarray(target, dtype=float).ravel()

        if t.shape != p.shape:
            if t.size == 1:
                t = np.full_like(p, t.item())
            else:
                raise ValueError("predicted and target shapes incompatible")

        per_elem = - (t * np.log(p) + (1.0 - t) * np.log(1.0 - p))

        if reduction == 'mean':
            return float(np.mean(per_elem))
        elif reduction == 'sum':
            return float(np.sum(per_elem))
        elif reduction == 'none':
            return per_elem
        else:
            raise ValueError("reduction must be 'mean', 'sum' or 'none'")


In [93]:
X = [14.1, 20.3, 0.095]
target_output = [1]


In [94]:
W1 = [[0.5, -0.3, 0.8],
      [0.2, 0.4, -0.6],
      [-0.7, 0.9, 0.1]]
B1 = [0.3, -0.5, 0.6]

layer1 = Dense_Layer(X, W1, B1, activation='relu')
A1 = layer1.forward()
Z1 = layer1.z
A1


array([ 1.336 , 10.383 ,  9.0095])

In [95]:
W2 = [[0.6, -0.2, 0.4],
      [-0.3, 0.5, 0.7]]
B2 = [0.1, -0.8]

layer2 = Dense_Layer(A1, W2, B2, activation='sigmoid')
A2 = layer2.forward()
Z2 = layer2.z
A2


array([0.91899725, 0.99996628])

In [96]:
W3 = [0.7, -0.5]
B3 = [0.2]

layer3 = Dense_Layer(A2, W3, B3, activation='sigmoid')
A3 = layer3.forward()
Z3 = layer3.z
A3


np.float64(0.5849955347015883)

In [97]:
loss = Dense_Layer.loss(A3, target_output, reduction='mean')
loss


0.53615106476815

In [98]:
# === Final summary cell (complete version) ===
import numpy as np

print("="*80)
print("COMPLETE FORWARD PASS SUMMARY")
print("="*80)
print(f"Input: {np.array(X)}")
print(f"Target: {np.array(target_output)}  (Malignant = 1, Benign = 0)")
print()

print(f"{'Layer':<15}{'Shape':<15}{'Activation':<15}{'Output Values'}")
print("-"*80)

# Input
print(f"{'Input':<15}{str(np.shape(X)):<15}{'None':<15}{np.round(np.array(X),6)}")

# Hidden 1
print(f"{'Hidden 1 Z1':<15}{str(np.shape(Z1)):<15}{'Linear':<15}{np.round(np.asarray(Z1),6)}")
print(f"{'Hidden 1 A1':<15}{str(np.shape(A1)):<15}{'ReLU':<15}{np.round(np.asarray(A1),6)}")

# Hidden 2
print(f"{'Hidden 2 Z2':<15}{str(np.shape(Z2)):<15}{'Linear':<15}{np.round(np.asarray(Z2),6)}")
print(f"{'Hidden 2 A2':<15}{str(np.shape(A2)):<15}{'Sigmoid':<15}{np.round(np.asarray(A2),6)}")

# Output
print(f"{'Output Z3':<15}{str(np.shape(Z3)):<15}{'Linear':<15}{np.round(np.asarray(Z3),6)}")
print(f"{'Output A3':<15}{str(np.shape(A3)):<15}{'Sigmoid':<15}{np.round(np.asarray(A3),6)}")

print()
print(f"{'Metrics':<25}{'Value'}")
print("-"*40)
pred_val = float(np.asarray(A3).ravel().item())
pred_class = 'Malignant (1)' if pred_val >= 0.5 else 'Benign (0)'
print(f"{'Predicted Class':<25}{pred_class}")
print(f"{'Confidence':<25}{pred_val:.6f}")
print(f"{'Binary Cross-Entropy Loss':<25}{loss:.6f}")
correct = (1 if pred_val >= 0.5 else 0) == int(np.asarray(target_output).ravel().item())
print(f"{'Correct Prediction':<25}{correct}")

print()
print("=== Architecture Summary ===")
print("Total Parameters:")
print(f"  - W1: {np.size(W1)}, B1: {np.size(B1)}")
print(f"  - W2: {np.size(W2)}, B2: {np.size(B2)}")
print(f"  - W3: {np.size(W3)}, B3: {np.size(B3)}")
total_params = np.size(W1) + np.size(B1) + np.size(W2) + np.size(B2) + np.size(W3) + np.size(B3)
print(f"  - Total: {total_params} parameters")
print("="*80)


COMPLETE FORWARD PASS SUMMARY
Input: [14.1   20.3    0.095]
Target: [1]  (Malignant = 1, Benign = 0)

Layer          Shape          Activation     Output Values
--------------------------------------------------------------------------------
Input          (3,)           None           [14.1   20.3    0.095]
Hidden 1 Z1    (3,)           Linear         [ 1.336  10.383   9.0095]
Hidden 1 A1    (3,)           ReLU           [ 1.336  10.383   9.0095]
Hidden 2 Z2    (2,)           Linear         [ 2.4288  10.29735]
Hidden 2 A2    (2,)           Sigmoid        [0.918997 0.999966]
Output Z3      ()             Linear         0.343315
Output A3      ()             Sigmoid        0.584996

Metrics                  Value
----------------------------------------
Predicted Class          Malignant (1)
Confidence               0.584996
Binary Cross-Entropy Loss0.536151
Correct Prediction       True

=== Architecture Summary ===
Total Parameters:
  - W1: 9, B1: 3
  - W2: 6, B2: 2
  - W3: 2, B3: 1
 

In [99]:
# === Hidden Layer 2 + Loss Summary ===
print("="*80)
print("HIDDEN LAYER 2 & LOSS DETAILS")
print("="*80)

print(f"{'Hidden Layer 2 Z2              ':<25}{np.round(np.asarray(Z2), 6)}")
print(f"{'Hidden Layer 2 A2 (Sigmoid)    ':<25}{np.round(np.asarray(A2), 6)}")
print()
print(f"{'Output Z3                      ':<25}{np.round(np.asarray(Z3), 6)}")
print(f"{'Output A3 (Sigmoid)            ':<25}{np.round(np.asarray(A3), 6)}")
print()
print(f"{'Binary Cross-Entropy Loss      ':<25}{loss:.6f}")
print("="*80)


HIDDEN LAYER 2 & LOSS DETAILS
Hidden Layer 2 Z2              [ 2.4288  10.29735]
Hidden Layer 2 A2 (Sigmoid)    [0.918997 0.999966]

Output Z3                      0.343315
Output A3 (Sigmoid)            0.584996

Binary Cross-Entropy Loss      0.536151
